# Advent of Code 2021 day 9

In [1]:
from collections import *
from itertools import *
from functools import *

from aocd.models import Puzzle
import numpy as np
import parse
from aocp import *

example: str = """2199943210
3987894921
9856789892
8767896789
9899965678"""
example_sol_a: int = 15
example_sol_b: int = 1134


puzzle = Puzzle(year=2021, day=9)
raw_data = puzzle.input_data

In [2]:
def parse_input(raw_data: str):
    return np.array(ListParser(ListParser(int)).parse(raw_data))

In [3]:
example_data = parse_input(example)
data = parse_input(raw_data)

In [4]:
example_data

array([[2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
       [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
       [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
       [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
       [9, 8, 9, 9, 9, 6, 5, 6, 7, 8]])

## Part 1

In [5]:
def get_diff_map(data: np.ndarray) -> np.ndarray:
    up   = np.concatenate((np.ones((1, data.shape[1]), dtype=int)*10, data[:-1, :] - data[1:, :]), axis=0)
    left = np.concatenate((np.ones((data.shape[0], 1), dtype=int)*10, data[:, :-1] - data[:, 1:]), axis=1)
    down = np.concatenate((data[1:, :] - data[:-1, :], np.ones((1, data.shape[1]), dtype=int)*10), axis=0)
    right = np.concatenate((data[:, 1:] - data[:, :-1], np.ones((data.shape[0], 1), dtype=int)*10), axis=1)
    return np.array([up, right, down, left])

In [6]:
get_diff_map(example_data)

array([[[10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [-1, -8,  1,  2,  1, -5, -1, -7, -1, -1],
        [-6,  1,  3,  1,  1,  1, -5,  1, -7, -1],
        [ 1,  1, -1, -1, -1, -1,  3,  1,  1, -7],
        [-1, -1, -3, -2, -1,  3,  1,  1,  1,  1]],

       [[-1,  8,  0,  0, -5, -1, -1, -1, -1, 10],
        [ 6, -1, -1,  1,  1, -5,  5, -7, -1, 10],
        [-1, -3,  1,  1,  1,  1, -1,  1, -7, 10],
        [-1, -1,  1,  1,  1, -3,  1,  1,  1, 10],
        [-1,  1,  0,  0, -3, -1,  1,  1,  1, 10]],

       [[ 1,  8, -1, -2, -1,  5,  1,  7,  1,  1],
        [ 6, -1, -3, -1, -1, -1,  5, -1,  7,  1],
        [-1, -1,  1,  1,  1,  1, -3, -1, -1,  7],
        [ 1,  1,  3,  2,  1, -3, -1, -1, -1, -1],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10]],

       [[10,  1, -8,  0,  0,  5,  1,  1,  1,  1],
        [10, -6,  1,  1, -1, -1,  5, -5,  7,  1],
        [10,  1,  3, -1, -1, -1, -1,  1, -1,  7],
        [10,  1,  1, -1, -1, -1,  3, -1, -1, -1],
        [10,  1, -1,  0,  0,  3,  1, -1, -1,

In [7]:
def solve_a(data) -> int:
    diff_map = get_diff_map(data)
    is_low_point = np.all(diff_map > 0, axis=0)
    return np.sum(data[is_low_point]+1)

In [8]:
solution_a = solve_a(data)
print(solution_a)

591


In [9]:
puzzle.answer_a = solution_a

Part a already solved with same answer: 591


## Part 2

In [10]:
def flows_towards(diff_map, pos):
    dir = np.argmin(diff_map[:, pos[0], pos[1]])
    if dir == 0:
        return (pos[0]-1, pos[1])
    elif dir == 1:
        return (pos[0], pos[1]+1)
    elif dir == 2:
        return (pos[0]+1, pos[1])
    elif dir == 3:
        return (pos[0], pos[1]-1)

In [11]:
diff_map = get_diff_map(example_data)
is_low_point = np.all(diff_map > 0, axis=0)

In [12]:
def get_flows_map(data: np.ndarray) -> np.ndarray:
    diff_map = get_diff_map(data)
    is_low_point = np.all(diff_map > 0, axis=0)
    points = np.argwhere(data>=0).tolist()
    lowpoints = np.argwhere(is_low_point).tolist()
    highpoints = np.argwhere(data==9).tolist()
    flowpoints = [point for point in points if point not in lowpoints and point not in highpoints]
    flows_mapping = {tuple(pos): flows_towards(diff_map, tuple(pos)) for pos in flowpoints}
    return flows_mapping

In [13]:
def get_lowpoint_mapping(flows_mapping: dict) -> dict:
    lowpoint_mapping = {}
    for pos, flow in flows_mapping.items():
        point = flow
        while True:
            if point not in flows_mapping:
                lowpoint_mapping[pos] = point
                break
            point = flows_mapping[point]
    return lowpoint_mapping

In [14]:
def solve_b(data) -> int:
    flows_mapping = get_flows_map(data)
    lowpoint_mapping = get_lowpoint_mapping(flows_mapping)
    _, counts = np.unique(np.array(list(lowpoint_mapping.values())), axis=0, return_counts=True)
    sizes = counts + 1
    return np.prod(sizes[sizes.argsort()[-3:]])

In [15]:
solve_b(example_data)

1134

In [16]:
solution_b = solve_b(data)
print(solution_b)

1113424


In [17]:
puzzle.answer_b = solution_b

Part b already solved with same answer: 1113424
